In [1]:
import pkg_resources
try:
    pkg_resources.require(['eth-ape','ape-alchemy','ape-etherscan','mplfinance', 'pandas', 'numpy', 'python-dotenv'])
    print('All packages installed')
except:
    !pip3 install eth-ape ape-alchemy ape-etherscan mplfinance pandas numpy python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.1/73.1 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 21.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 27.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.3/98.3 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 32.7 MB/s eta 0:00:00


In [2]:
import matplotlib.pyplot as plt
import mplfinance as mpf
import numpy as np
import pandas as pd
import typing as tp
from ape import accounts, chain, Contract, networks

/Users/handu/opt/anaconda3/envs/crypto/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
# manually input your keys like this
# %env WEB3_ALCHEMY_API_KEY = xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
# %env ETHERSCAN_API = xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx

# or load from a separate ".env" file, which is more secure!
from dotenv import load_dotenv
load_dotenv()

True

In [29]:
context = networks.parse_network_choice('ethereum:mainnet:https://mainnet.infura.io/v3/a00555d697a0423785aa7a966d88701a')
context.__enter__()
print(f"ape version: {pkg_resources.get_distribution('eth-ape').version}")
print(f"provider.config of type={type(context.provider.config)}")
for k,v in context.provider.config.Config.__dict__.items():
    if not k.startswith('__'):
        print(f" {k} = {v}")


INFO: Connecting to existing Geth node at  https://mainnet.infura.io/v3/a00555d697a0423785aa7a966d88701a.
ape version: 0.6.8
provider.config of type=<class 'ape_geth.provider.GethConfig'>
 extra = allow


In [30]:
for plugin_name, ecosystem_class in context.plugin_manager.providers:
    print(f"plugin_name={plugin_name}, ecosystem_class={ecosystem_class}")

plugin_name=alchemy, ecosystem_class=('ethereum', 'mainnet', <class 'ape_alchemy.provider.Alchemy'>)
plugin_name=alchemy, ecosystem_class=('ethereum', 'goerli', <class 'ape_alchemy.provider.Alchemy'>)
plugin_name=alchemy, ecosystem_class=('ethereum', 'sepolia', <class 'ape_alchemy.provider.Alchemy'>)
plugin_name=alchemy, ecosystem_class=('arbitrum', 'mainnet', <class 'ape_alchemy.provider.Alchemy'>)
plugin_name=alchemy, ecosystem_class=('arbitrum', 'goerli', <class 'ape_alchemy.provider.Alchemy'>)
plugin_name=alchemy, ecosystem_class=('optimism', 'mainnet', <class 'ape_alchemy.provider.Alchemy'>)
plugin_name=alchemy, ecosystem_class=('optimism', 'goerli', <class 'ape_alchemy.provider.Alchemy'>)
plugin_name=alchemy, ecosystem_class=('polygon', 'mainnet', <class 'ape_alchemy.provider.Alchemy'>)
plugin_name=alchemy, ecosystem_class=('polygon', 'mumbai', <class 'ape_alchemy.provider.Alchemy'>)
plugin_name=test, ecosystem_class=('ethereum', 'local', <class 'ape_test.provider.LocalProvider'>

In [31]:
# get account 0
dev = accounts.test_accounts[0]
dev

<TestAccount 0x1e59ce931B4CFea3fe4B875411e280e173cB7A9C>

In [32]:
# get the balance of account 0
dev.balance

0

In [33]:
current_block = chain.blocks[-1].number
current_block

17067508

In [34]:
# determine starting block for the query
dblocks = int(86400*90/12) # last 180 d
start_block = current_block - dblocks
start_block

16419508

In [35]:
curve_steth = Contract("0xDC24316b9AE028F1497c275EB9192a3Ea0f67022")
curve_steth

<Vyper_contract 0xDC24316b9AE028F1497c275EB9192a3Ea0f67022>

In [36]:
steth = Contract("0xae7ab96520DE3A18E5e111B5EaAb095312D7fE84")
steth

<Lido 0xae7ab96520DE3A18E5e111B5EaAb095312D7fE84>

In [37]:
steth.balanceOf(curve_steth)

393710999537239375481237

In [38]:
curve_steth.TokenExchange

TokenExchange(address indexed buyer, int128 sold_id, uint256 tokens_sold, int128 bought_id, uint256 tokens_bought)

In [39]:
type(curve_steth.TokenExchange)

ape.contracts.base.ContractEvent

In [40]:
%time query_swaps_recent = curve_steth.TokenExchange.query("*", start_block=current_block-3000) # last 10 hr
query_swaps_recent

CPU times: user 86.6 ms, sys: 8.67 ms, total: 95.3 ms
Wall time: 409 ms


,event_name,contract_address,event_arguments,transaction_hash,block_number,block_hash,log_index,transaction_index
0,TokenExchange,0xDC24316b9AE028F1497c275EB9192a3Ea0f67022,{'buyer': '0x00Df529CD3dc337761075FEdBA4576FdA...,0xb801a838cf749f743c26ff0c710344ff4cbcf37b8a0c...,17064624,0x66af28b5250df068968263e72462cf70ec913c95c714...,296,137
1,TokenExchange,0xDC24316b9AE028F1497c275EB9192a3Ea0f67022,{'buyer': '0x99a58482BD75cbab83b27EC03CA68fF48...,0x0bd4c15e0a3bee51c0a4932491aa10d076d800b509c4...,17064656,0x36f52de51ba8984ceecd53de937c8b5996da65c78dc8...,37,17
2,TokenExchange,0xDC24316b9AE028F1497c275EB9192a3Ea0f67022,{'buyer': '0xEf0D72C594b28252BF7Ea2bfbF0987924...,0x325a153ec8e0329e29a62ecf2cd427e1a096aaff71b5...,17064744,0x647da2b08881464ba967f61cd4fcd21266b596b4a9e8...,181,102
3,TokenExchange,0xDC24316b9AE028F1497c275EB9192a3Ea0f67022,{'buyer': '0x99a58482BD75cbab83b27EC03CA68fF48...,0x6caf0cd26ebb866cc58bb45b35a3c97ac32551af93b3...,17064750,0x7a0ca8b756989dedbfecd0bf10572f8fd998d9a7dfb7...,116,14
4,TokenExchange,0xDC24316b9AE028F1497c275EB9192a3Ea0f67022,{'buyer': '0x357b10A60fd902feDBbCf6492734937Df...,0xe6a9fb62d27e295aea253333c3e7c996219b914c0a8c...,17064764,0x0999ef10f4c2985a6b216161bfc2342e69f67fc63fce...,124,60
5,TokenExchange,0xDC24316b9AE028F1497c275EB9192a3Ea0f67022,{'buyer': '0x99a58482BD75cbab83b27EC03CA68fF48...,0x46537d78c1fdd655ff555c93555cd6025f9b240f33a6...,17064775,0x692e5e1eb1200adea2014ec4f97891835e20848ff1a1...,44,22
6,TokenExchange,0xDC24316b9AE028F1497c275EB9192a3Ea0f67022,{'buyer': '0x22F9dCF4647084d6C31b2765F6910cd85...,0xbf2e711a0e3f82beac14c969130c89eccbd43029b9ac...,17064791,0xa283dbbdef05ce2bfbde38081e3a604d6f6fb5c69dd3...,354,131
7,TokenExchange,0xDC24316b9AE028F1497c275EB9192a3Ea0f67022,{'buyer': '0x22F9dCF4647084d6C31b2765F6910cd85...,0x6a2214c351c2ceab2ee8dcd00dcfc715f150198edba1...,17064920,0xf9e0551ea7ad4437f5bcfa5f99f48b9f0ccfa53e96a2...,111,36
8,TokenExchange,0xDC24316b9AE028F1497c275EB9192a3Ea0f67022,{'buyer': '0x1136B25047E142Fa3018184793aEc68fB...,0x30747dc41d4f4ad5e3dcb3bf35bb214589046e01de81...,17064957,0x509f6af19e890fa632da5a1a3400284fa2b80559fe7b...,274,102
9,TokenExchange,0xDC24316b9AE028F1497c275EB9192a3Ea0f67022,{'buyer': '0xf6DEA4f5CCAD6582dfd97799f1c7941a2...,0xd45d329646681e2c3f7b3880e5ad60c7ab091047bfbc...,17064998,0x1786e915b07b288efd0da9520a620dda1f542c5dfb72...,285,82


In [41]:
# sweet, very cool :)
type(query_swaps_recent)

pandas.core.frame.DataFrame

In [42]:
# take a deeper dive into event arguments column. it's a pandas series
query_swaps_recent["event_arguments"][:10]

0    {'buyer': '0x00Df529CD3dc337761075FEdBA4576FdA...
1    {'buyer': '0x99a58482BD75cbab83b27EC03CA68fF48...
2    {'buyer': '0xEf0D72C594b28252BF7Ea2bfbF0987924...
3    {'buyer': '0x99a58482BD75cbab83b27EC03CA68fF48...
4    {'buyer': '0x357b10A60fd902feDBbCf6492734937Df...
5    {'buyer': '0x99a58482BD75cbab83b27EC03CA68fF48...
6    {'buyer': '0x22F9dCF4647084d6C31b2765F6910cd85...
7    {'buyer': '0x22F9dCF4647084d6C31b2765F6910cd85...
8    {'buyer': '0x1136B25047E142Fa3018184793aEc68fB...
9    {'buyer': '0xf6DEA4f5CCAD6582dfd97799f1c7941a2...
Name: event_arguments, dtype: object

In [43]:
type(query_swaps_recent["event_arguments"])

pandas.core.series.Series

In [44]:
query_swaps_recent["event_arguments"].iloc[-1]

{'buyer': '0x1136B25047E142Fa3018184793aEc68fBB173cE4',
 'sold_id': 1,
 'tokens_sold': 2000000000000000001,
 'bought_id': 0,
 'tokens_bought': 1998309472753507351}

In [45]:
# ok great. let's look a bit further into the past to start_block from above. see how long it takes
%time query_swaps_original = curve_steth.TokenExchange.query("*", start_block=start_block)
query_swaps_original

CPU times: user 4.41 s, sys: 242 ms, total: 4.65 s
Wall time: 5.31 s


,event_name,contract_address,event_arguments,transaction_hash,block_number,block_hash,log_index,transaction_index
0,TokenExchange,0xDC24316b9AE028F1497c275EB9192a3Ea0f67022,{'buyer': '0x6A08E13a340Bf901B1625E3e2fd471fEf...,0xfaaade56c19b1a3f84d24e81da8f265da65f49895ef7...,16419539,0xa2b704c860343aef60e0f4852dd17cdf6d113d2ddca7...,145,76
1,TokenExchange,0xDC24316b9AE028F1497c275EB9192a3Ea0f67022,{'buyer': '0x22F9dCF4647084d6C31b2765F6910cd85...,0xdfb4ddc3f3e5ccbe1f515e0f30e352aa985f04c0ffaf...,16419576,0x91233f8bd289f7bc07d854da08d2f9f16cd48a3f8e3b...,85,53
2,TokenExchange,0xDC24316b9AE028F1497c275EB9192a3Ea0f67022,{'buyer': '0x99a58482BD75cbab83b27EC03CA68fF48...,0x59cd7a7282f901b47bce0852c70200e0f4c5503ae253...,16419593,0xca03b6456f7b08708ef8f70a9189a9b894efad3fa051...,14,4
3,TokenExchange,0xDC24316b9AE028F1497c275EB9192a3Ea0f67022,{'buyer': '0x22F9dCF4647084d6C31b2765F6910cd85...,0x6be0c0ce2f4dc27d1a47fedc0d5adf522f318f5d1239...,16419609,0x9a4f133d31a82511d94ea52eecad498e2b5276f6ed68...,69,32
4,TokenExchange,0xDC24316b9AE028F1497c275EB9192a3Ea0f67022,{'buyer': '0x99a58482BD75cbab83b27EC03CA68fF48...,0x62c5f2e331d1e7afdea2c5908f4322ddb2677e0da65d...,16419615,0x38f93d16eb4d1ad1c8f211a7e92688402f25b1dacea4...,47,22
...,...,...,...,...,...,...,...,...
13255,TokenExchange,0xDC24316b9AE028F1497c275EB9192a3Ea0f67022,{'buyer': '0xe4614d81Bd262D62b09867c915d556c50...,0xffc7630a6cb5edfa31f28642a77f1ec5d5b73d0db0fb...,17067168,0xcf59d0d6742ba54ac0b6896e4add1c894d3bfa5bd43b...,96,44
13256,TokenExchange,0xDC24316b9AE028F1497c275EB9192a3Ea0f67022,{'buyer': '0x69cE8721790Edbdcd2b4155D853d99d26...,0xbaac688eae191dc3a1be81df35ed489f0474302fd081...,17067244,0x14ad419ae258431cdc469444530c6463358fe5410360...,145,81
13257,TokenExchange,0xDC24316b9AE028F1497c275EB9192a3Ea0f67022,{'buyer': '0xd669621C0F97B11f8E74Fa57bBF7e33dF...,0xfb33b672ee9b0d75dab8b07b1287149be1025ddde436...,17067297,0x8a7c712ea7f6e43568380aa09d61f358e67b2c972121...,353,171
13258,TokenExchange,0xDC24316b9AE028F1497c275EB9192a3Ea0f67022,{'buyer': '0x99a58482BD75cbab83b27EC03CA68fF48...,0xfe7706b3e574315b0695421b1a6ce36c9279cd9bb23c...,17067431,0x2a2ddf3419e41ecfee8ec9dd5d6ef3f0296f76c662ed...,296,168


In [46]:
query_swaps = query_swaps_original.copy()

In [47]:
# get coin indices:
#  - 0 = ETH (native)
#  - 1 = stETH
coin_indices = [curve_steth.coins(i) for i in range(2)]
coin_indices

['0xEeeeeEeeeEeEeeEeEeEeeEEEeeeeEeeeeeeeEEeE',
 '0xae7ab96520DE3A18E5e111B5EaAb095312D7fE84']

In [48]:
steth.address == coin_indices[-1]

True

In [49]:
# cache the fee to calculate associated price correctly (prior to fees applied)
curve_steth_fee = curve_steth.fee() / 1e10  # fee denom is 1e10
curve_steth_fee

0.0004

In [50]:
# if sold_id == 1, then selling stETH for ETH, which is price wanted
#  i.e. quote = <ETH>, base = <stETH>
last_swap = query_swaps["event_arguments"].iloc[-1]
print('last swap event:', last_swap)

last_price = last_swap['tokens_bought'] * (1+curve_steth_fee) / last_swap['tokens_sold']
if last_swap['sold_id'] == 0:
    last_price = 1 / last_price
print('price:', last_price)

last swap event: {'buyer': '0x1136B25047E142Fa3018184793aEc68fBB173cE4', 'sold_id': 1, 'tokens_sold': 2000000000000000001, 'bought_id': 0, 'tokens_bought': 1998309472753507351}
price: 0.9995543982713043


In [54]:
# write a function that iterates over event_arguments to calculate marginal price from last token swap
def get_price(event_args: dict) -> float:
    try:
        price = event_args['tokens_bought'] * (1+curve_steth_fee) / event_args['tokens_sold']
    except:
        price = -1
        pass
    if event_args['sold_id'] == 0:
        try: 
            price = 1 / price
        except:
            price = -1
            pass
    return price

In [55]:
# validate with logic in cell above for last price
get_price(query_swaps["event_arguments"].iloc[-1]) == last_price

True

In [56]:
# create new column with marginal price (stETH as base and ETH as quote, so should be at a discount)
query_swaps['price (STETH/ETH)'] = query_swaps['event_arguments'].apply(get_price)
query_swaps

ZeroDivisionError: float division by zero

In [ ]:
query_swaps.plot(x='block_number', y='price (STETH/ETH)')

In [ ]:
# well those are weird dips and spikes. look at smooth parts
query_swaps[(query_swaps['block_number'] > 15100000) & (query_swaps['block_number'] < 15200000)].plot(x='block_number', y='price (STETH/ETH)')

In [ ]:
# weird spike for swaps back to 1.000 exchange rate. should look at this in event df
query_swaps[(query_swaps['block_number'] > 15131500) & (query_swaps['block_number'] < 15133000)].plot(x='block_number', y='price (STETH/ETH)')

In [ ]:
# first, unfold event arguments into separate columns
def unfold_event_args(key: str, event_args: dict):
    return event_args[key]

for key in query_swaps.iloc[-1]['event_arguments'].keys():
    query_swaps[key] = query_swaps['event_arguments'].apply(lambda ev: unfold_event_args(key, ev))

In [ ]:
query_swaps

In [ ]:
# now, check swaps where price spikes up to 1 between blocks 15132100 and 15132400
query_swaps[(query_swaps['block_number'] > 15132100) & (query_swaps['block_number'] < 15132400)]

In [ ]:
# ahh, ok :) .. someones selling 1 wei into the pool (see e.g. block 15132225)
# which is causing rounding issues for price calc

In [ ]:
# are the dusting rounding issues also causing those spikes down to zero?
# look at first spike down to 0
query_swaps[(query_swaps['block_number'] > 14824600) & (query_swaps['block_number'] < 14825000)].plot(x='block_number', y='price (STETH/ETH)')

In [ ]:
# looks like it (see swap at block 14824840); similar rounding issues except rounded down to zero here
query_swaps[(query_swaps['block_number'] > 14824600) & (query_swaps['block_number'] < 14825000)]

In [ ]:
# apply min size to swaps we care about to ignore the dustings
min_bought = 0.0001 * 1e18  # trade size must be > 0.0001 ETH 
query_swaps = query_swaps[query_swaps['tokens_bought'] > min_bought]

In [ ]:
# nice! way better
query_swaps.plot(x='block_number', y='price (STETH/ETH)')

In [ ]:
# should smooth the price data out so not so jittery by averaging over 1h for hourly candles.
# keep the volumes numbers as well, so make some slight changes to the (tokens_bought, tokens_sold)
# columns to instead track (volume_coin0, volume_coin1) at each swap

In [ ]:
# write a function that returns volume on coin indices for a swap (normalized by 1e18 for decimals)
def get_volume(coin_index: int, x: pd.Series) -> (float, float):
    volume = x['tokens_sold'] if x['sold_id'] == coin_index else x['tokens_bought']
    volume = volume / 1e18
    return volume

In [ ]:
%%capture
for coin_index in range(2):
    key = f'volume_coin{coin_index}'
    query_swaps[key] = query_swaps.apply(lambda x: get_volume(coin_index, x), axis=1)

In [ ]:
query_swaps

In [ ]:
# prep for smoothing by setting index to block number (choose last swap in block to represent price at block)
query_swaps_filtered = query_swaps.filter(['block_number', 'price (STETH/ETH)', 'volume_coin0', 'volume_coin1']).drop_duplicates('block_number', keep='last')
query_swaps_filtered = query_swaps_filtered.set_index('block_number')

In [ ]:
# reindex by block number (1 block steps) and ffill price values 
new_index = range(query_swaps_filtered.index.min(), query_swaps_filtered.index.max()+1)
query_swaps_filtered = query_swaps_filtered.reindex(new_index).ffill()

In [ ]:
# separate data into windows of 260 blocks (~ 1h)
# remove first (last_block - first_block) % window rows so even candle windows
window = 260
first_index = (query_swaps_filtered.index[-1] - query_swaps_filtered.index[0]) % window
query_swaps_filtered = query_swaps_filtered[first_index:]

In [ ]:
# smooth data using mean over 260 blocks for ~ 1h candles
s = (query_swaps_filtered.index.to_series() / window).astype(int)
query_swaps_candled = query_swaps_filtered.groupby(s).agg({'price (STETH/ETH)': ['first', 'max', 'min', 'last', 'mean'], 'volume_coin0': 'sum', 'volume_coin1': 'sum'}).set_index(s.index[::window])
query_swaps_candled

In [ ]:
# make some fancy plots w subplots ...
fig, ax = plt.subplots(2, 1)
query_swaps.plot(x='block_number', y='price (STETH/ETH)', ax=ax[0])
query_swaps_candled['price (STETH/ETH)'].plot(y='mean', ax=ax[0], label='price (STETH/ETH) - 1H mean')
query_swaps_candled['volume_coin0'].plot(y='sum', color='g', ax=ax[1], label='volume (ETH)')

In [ ]:
# plot the candles since that's dope :o

In [ ]:
# get the timestamps every 24 candles (~ 24h) and interpolate the rest (SEE banteg gist again)
# NOTE: include last block in index for interp to work properly
T = 24
blocks = query_swaps_candled.index[::24].to_list() + [query_swaps_candled.index[-1]]

In [ ]:
%time timestamps = [ chain.blocks[i].timestamp for i in blocks ]

In [ ]:
query_swaps_candled['timestamp'] = pd.to_datetime(np.interp(query_swaps_candled.index, blocks, timestamps) * 1e9)
query_swaps_candled

In [ ]:
# put data into new df for ohlcv with 1h candles
query_swaps_ohlcv = pd.DataFrame({
    'Open': query_swaps_candled['price (STETH/ETH)']['first'],
    'High': query_swaps_candled['price (STETH/ETH)']['max'],
    'Low': query_swaps_candled['price (STETH/ETH)']['min'],
    'Close': query_swaps_candled['price (STETH/ETH)']['last'],
    'Volume': query_swaps_candled['volume_coin0']['sum'],
    'Date': query_swaps_candled['timestamp']
})
query_swaps_ohlcv = query_swaps_ohlcv.set_index('Date')
query_swaps_ohlcv

In [ ]:
# oooooo fancy :)
def fancy_plot(
    data,
    mav=(10, 50),
    plt_type='candle',
    volume=True,
    datetime_format='%b%d',
    xrotation=0,
    tight_layout=True,
    style='mike', # mpf.available_styles() to pick your fav
    **kwargs
):
    mpf.plot(
        data,
        warn_too_much_data=2500,
        mav=mav,
        type=plt_type,
        volume=volume,
        datetime_format=datetime_format,
        xrotation=xrotation,
        tight_layout=tight_layout,
        style=style,
        **kwargs
    )

title="STETH/ETH"
fancy_plot(query_swaps_ohlcv, mav=(10, 50, 200), title=title)


In [ ]:
def plot_last_n_days(data, num_days_back=100, **kwargs):
    endPeriod = query_swaps_ohlcv.index[-1]
    startPeriod = endPeriod - pd.Timedelta(days=num_days_back)
    dataToPlot = data.loc[(data.index > startPeriod) & (data.index < endPeriod),:]
    title=f"STETH/ETH last {num_days_back} days"
    title+=f"({startPeriod:%b%d}-{endPeriod:%b%d})" if startPeriod.month != endPeriod.month else f"({startPeriod:%b%d}-{endPeriod:%d})"
    mav = [10]
    if num_days_back > 3:
        mav.append(50)
    if num_days_back > 10:
        mav.append(100)
    fancy_plot(dataToPlot, mav=mav, title=title)

plot_last_n_days(query_swaps_ohlcv, num_days_back=8)
plot_last_n_days(query_swaps_ohlcv, num_days_back=3)
